# Himmelblau Benchmark
## Imports

In [2]:
from bofire.benchmarks.single import Himmelblau
from bofire.runners.api import run
from bofire.data_models.strategies.api import SoboStrategy, RandomStrategy, PolytopeSampler
from bofire.data_models.acquisition_functions.api import qEI
import bofire.strategies.api as strategies
from bofire.data_models.api import Domain
from functools import partial
import pandas as pd
import os

SMOKE_TEST = os.environ.get("SMOKE_TEST")

## Random Optimization

In [3]:
def sample(domain):
    datamodel = PolytopeSampler(domain=domain)
    sampler = strategies.map(data_model=datamodel)
    sampled = sampler.ask(10)
    return sampled

def best(domain: Domain, experiments: pd.DataFrame) -> float:
    return experiments.y.min()

random_results = run(
    Himmelblau(),
    strategy_factory=lambda domain: strategies.map(RandomStrategy(domain=domain)),
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

run 00 with current best 4.352: 100%|██████████████████████████████████████████████████| 50/50 [00:01<00:00, 28.18it/s]


## SOBO (GPEI) Optimization

In [5]:
def strategy_factory(domain: Domain):
    data_model = SoboStrategy(domain=domain, acquisition_function=qEI())
    return strategies.map(data_model)

bo_results = run(
    Himmelblau(),
    strategy_factory=strategy_factory,
    n_iterations=50 if not SMOKE_TEST else 1,
    metric=best,
    initial_sampler=sample,
    n_runs=1,
    n_procs=1,
)

run 00 with current best 0.003: 100%|██████████████████████████████████████████████████| 50/50 [07:19<00:00,  8.79s/it]
